In [34]:
%%time
import pal
import json
import random
import io
import pandas as pd
from pal.prompt import math_prompts, colored_object_prompt, penguin_prompt, date_understanding_prompt, algorithmic_prompt
from tqdm.notebook import tqdm

CPU times: user 28 μs, sys: 4 μs, total: 32 μs
Wall time: 36.5 μs


In [44]:
import pandas as pd

def CREATE(table, action):
    expression = action[action.find('CREATE_START[') + len('CREATE_START['):action.find(']CREATE_END')]
    try:
        data = json.loads(expression)
        table = pd.DataFrame(data)
        print(table)
        return "Created table"
    except:
        return "That was not a valid input. Try again."
curTable = pd.DataFrame()
evaled = CREATE(curTable, """CREATE_START[{"name": ["Louis", "Bernard", "Vincent", "Gwen"], "age": [7, 5, 9, 8], "height": [50, 80, 60, 70], "weight": [11, 13, 11, 15]}]CREATE_END""")
print(curTable)

      name  age  height  weight
0    Louis    7      50      11
1  Bernard    5      80      13
2  Vincent    9      60      11
3     Gwen    8      70      15
Empty DataFrame
Columns: []
Index: []


In [35]:
create_function_prompt = """What other action should this library have? Remember: First, decide on an appropriate action. Write instructions for this action, by putting it in between INSTRUCTIONS_START and INSTRUCTIONS_END. Then, write python code defining a function that takes in the pandas Dataframe 'table' and the string 'action' as input and returns the desired output. Only write one additional action. If you think there are no further actions that need to be written, write 'FINISHED'. Do not repeat actions."""
create_function_example = [
    {"role": "user", "content": """I want to create a library for Tables that contains actions. First, decide on an appropriate action. Write instructions for this action, by putting it in between INSTRUCTIONS_START and INSTRUCTIONS_END. Then, write python code defining a function that takes in the pandas Dataframe 'table' and the string 'action' as input and returns the desired output."""},
    {"role": "assistant", "content": """I will create an action called ADD, which takes in any amount of numbers, separated by semicolons, and returns their sum.
I first write an instruction for this.
INSTRUCTIONS_START
CREATE_START[data]CREATE_END: CREATE takes a data definition of a table which will be used to generate a pandas dataframe, so make sure that the input does not contain anything unnecessary for the pandas dataframe input generation. Write the data in json format. For example, for this example table:
name, age, height (cm), weight (kg) 
Feynman, 43, 25, 69
Oppie, 28, 73, 42
Bob, 11, 23, 83
you would write: CREATE_START[{"name": ["Louis", "Bernard", "Vincent", "Gwen"], "age": [7, 5, 9, 8], "height": [50, 80, 60, 70], "weight": [11, 13, 11, 15]}]CREATE_END
INSTRUCTIONS_END 
Now I write python code to define a function ADD that takes in a string called "action", extracts the expression inside the ADD_START[]ADD_END, and returns the correct value.
```python
import pandas as pd

def CREATE(table, action):
    expression = action[action.find('CREATE_START[') + len('CREATE_START['):action.find(']CREATE_END')]
    try:
        data = json.loads(expression)
        table = pd.DataFrame(data)
        return "Created table"
    except:
        return "That was not a valid input. Try again."
```
"""},
    {"role": "user", "content": create_function_prompt}
]

In [36]:
interface = pal.interface.ProgramInterface(
  model='gpt-4o-mini',
  stop='\n\n\n', # stop generation str for Codex API
  get_answer_expr='solution()' # python expression evaluated after generated code to obtain answer 
)
max_count = 10
libraryFunctions = {
    "CREATE": [
        """
CREATE_START[data]CREATE_END: CREATE takes a data definition of a table which will be used to generate a pandas dataframe, so make sure that the input does not contain anything unnecessary for the pandas dataframe input generation. Write the data in json format. For example, for this example table:
name, age, height (cm), weight (kg) 
Feynman, 43, 25, 69
Oppie, 28, 73, 42
Bob, 11, 23, 83
you would write: CREATE_START[{"name": ["Louis", "Bernard", "Vincent", "Gwen"], "age": [7, 5, 9, 8], "height": [50, 80, 60, 70], "weight": [11, 13, 11, 15]}]CREATE_END
""".strip(),
        """
import pandas as pd

def CREATE(table, action):
    expression = action[action.find('CREATE_START[') + len('CREATE_START['):action.find(']CREATE_END')]
    try:
        data = json.loads(expression)
        table = pd.DataFrame(data)
        return "Created table"
    except:
        return "That was not a valid input. Try again."
""".strip()
    ]
}
all_instructions = ["""
CREATE_START[data]CREATE_END: CREATE takes a data definition of a table which will be used to generate a pandas dataframe, so make sure that the input does not contain anything unnecessary for the pandas dataframe input generation. Write the data in json format. For example, for this example table:
name, age, height (cm), weight (kg) 
Feynman, 43, 25, 69
Oppie, 28, 73, 42
Bob, 11, 23, 83
you would write: CREATE_START[{"name": ["Louis", "Bernard", "Vincent", "Gwen"], "age": [7, 5, 9, 8], "height": [50, 80, 60, 70], "weight": [11, 13, 11, 15]}]CREATE_END
""".strip()]

for i in range(max_count):
    messages = []
    for message in create_function_example:
        messages.append(message)
    if len(all_instructions) != 0:
        new_prompt = "You have already made these actions:\n"
        for i in range(len(all_instructions)):
            new_prompt += '(' + str(i + 1) + ") "
            new_prompt += all_instructions[i] 
            new_prompt += '\n'
        #print("NEW PROMPT")
        #print(new_prompt)
        #print("---------------------------------------")
        messages.append({"role": "user", "content": new_prompt})
    gens = interface.generate_history(messages=messages, max_tokens=1024)
    output = gens[0]
    print("OUTPUT:")
    print(output)
    print("######################################")
    if "FINISHED" in output:
        break
    curInstruct = output[output.find("INSTRUCTIONS_START") + len("INSTRUCTIONS_START") + 1:output.find("INSTRUCTIONS_END")].strip()
    all_instructions.append(curInstruct)
    pyStart = output.find("```python") + len("```python")
    pyEnd = None
    for i in range(len(output)):
        if output[i:i + 3] == "```":
            pyEnd = i
    pyCode = output[pyStart:pyEnd]
    libraryFunctions[pyCode[pyCode.find("def ") + len("def "):pyCode.find('(')].strip()] = [curInstruct, output[pyStart:pyEnd]]
    messages.append({"role": "assistant", "content": output})
all_instructions.append("FINISH_START[answer]FINISH_END: When you finish a question, write your answer as FINISH_START[answer]FINISH_END and terminate your response.")
"""
json_list = None
max_count = 1
counter = 0
responses = []
with open('datasets/reasoning_about_colored_objects.json', 'r') as json_file:
    json_list = json.load(json_file)
json_list = json_list["examples"]
jsonSamples = random.sample(range(0, len(json_list)), max_count)
print(jsonSamples)
for json_pos in tqdm(jsonSamples, desc="Going through Colored Object questions"):
    result = json_list[json_pos]
    question = result["input"]
    target = result["target_scores"]
    curResponse = {"question": question, "target": target}
    messages = []
"""

OUTPUT:
I will create an action called "SUM_COLUMN", which calculates the sum of a specified column in the DataFrame.

INSTRUCTIONS_START
SUM_COLUMN[column_name]: This action takes the name of a column in the DataFrame and returns the sum of all the values in that column. Ensure that the column name provided exists in the DataFrame. For example, if you have a DataFrame with a column "age", you would write: SUM_COLUMN[age].
INSTRUCTIONS_END

Now I will write the Python code to define the function that performs this action.

```python
import pandas as pd

def SUM_COLUMN(table, action):
    column_name = action[action.find('SUM_COLUMN[') + len('SUM_COLUMN['):action.find(']')]
    if column_name in table.columns:
        return table[column_name].sum()
    else:
        return f"Column '{column_name}' does not exist in the table."
```
######################################
OUTPUT:
I will create an action called AVERAGE_COLUMN, which calculates the average of the values in a specified colum

'\njson_list = None\nmax_count = 1\ncounter = 0\nresponses = []\nwith open(\'datasets/reasoning_about_colored_objects.json\', \'r\') as json_file:\n    json_list = json.load(json_file)\njson_list = json_list["examples"]\njsonSamples = random.sample(range(0, len(json_list)), max_count)\nprint(jsonSamples)\nfor json_pos in tqdm(jsonSamples, desc="Going through Colored Object questions"):\n    result = json_list[json_pos]\n    question = result["input"]\n    target = result["target_scores"]\n    curResponse = {"question": question, "target": target}\n    messages = []\n'

In [37]:
for instruct in all_instructions:
    print(instruct)
    print("")

CREATE_START[data]CREATE_END: CREATE takes a data definition of a table which will be used to generate a pandas dataframe, so make sure that the input does not contain anything unnecessary for the pandas dataframe input generation. Write the data in json format. For example, for this example table:
name, age, height (cm), weight (kg) 
Feynman, 43, 25, 69
Oppie, 28, 73, 42
Bob, 11, 23, 83
you would write: CREATE_START[{"name": ["Louis", "Bernard", "Vincent", "Gwen"], "age": [7, 5, 9, 8], "height": [50, 80, 60, 70], "weight": [11, 13, 11, 15]}]CREATE_END

SUM_COLUMN[column_name]: This action takes the name of a column in the DataFrame and returns the sum of all the values in that column. Ensure that the column name provided exists in the DataFrame. For example, if you have a DataFrame with a column "age", you would write: SUM_COLUMN[age].

AVERAGE_COLUMN[column_name]: This action takes the name of a column in the DataFrame and returns the average of all the values in that column. Ensure 

In [38]:
for code in libraryFunctions.values():
    print("CODE:")
    print(code[1])
    print("######################################\n")

CODE:
import pandas as pd

def CREATE(table, action):
    expression = action[action.find('CREATE_START[') + len('CREATE_START['):action.find(']CREATE_END')]
    try:
        data = json.loads(expression)
        table = pd.DataFrame(data)
        return "Created table"
    except:
        return "That was not a valid input. Try again."
######################################

CODE:

import pandas as pd

def SUM_COLUMN(table, action):
    column_name = action[action.find('SUM_COLUMN[') + len('SUM_COLUMN['):action.find(']')]
    if column_name in table.columns:
        return table[column_name].sum()
    else:
        return f"Column '{column_name}' does not exist in the table."

######################################

CODE:

import pandas as pd

def AVERAGE_COLUMN(table, action):
    column_name = action[action.find('AVERAGE_COLUMN[') + len('AVERAGE_COLUMN['):action.find(']')]
    if column_name in table.columns:
        return table[column_name].mean()
    else:
        return f"Column

In [39]:
ask_prompt = """Whenever you synthesize information and perform reasoning, it is a Thought. When you need to perform an operation on a table, it is an Action. There are """ + str(len(all_instructions)) + " actions:\n"
for i in range(len(all_instructions)):
    ask_prompt += '(' + str(i + 1) + ") "
    ask_prompt += all_instructions[i] 
    ask_prompt += '\n'
ask_prompt += """Terminate your response after an action. Then, an Observation is returned by the user. IMPORTANT: YOU DO NOT WRITE OBSERVATIONS. THEY WILL BE PROVIDED TO YOU. You will be given a question and you will perform Thoughts and Actions until you get the answer. When you get the answer, you will perform the Action called FINISH. While you are reasoning out solutions, you will continually refer back to relevant portions of the question and employ logical reasoning to determine your next action."""

In [40]:
json_list = []
max_count = 1
counter = 0
responses = []
with open('datasets/penguins_in_a_table.json', 'r') as json_file:
    allEntries = json.load(json_file)
    task = allEntries["task_prefix"]
    allEntries = allEntries["examples"]
    for entry in allEntries:
        entry["input"] = task + '\n' + entry["input"]
        json_list.append(entry)
jsonSamples = random.sample(range(0, len(json_list)), max_count)
maxIters = 20
print(jsonSamples)

for json_pos in tqdm(jsonSamples, desc="Going through math questions"):
    result = json_list[json_pos]
    #question = "Toulouse has twice as many sheep as Charleston. Charleston has 4 times as many sheep as Seattle. How many sheep do Toulouse, Charleston, and Seattle have together if Seattle has 20 sheep?"
    question = result["input"]
    target = result["target"]
    curResponse = {"question": question, "target": target}
    finished = False
    messages = [
        {"role": "system", "content": "You are a helpful assistant that accurately solves reasoning questions about tables by acting as an agent who will perform Thoughts and Actions."},
        {"role": "user", "content": ask_prompt + '\n' + """Question: Make the following table:
name, age, height (cm), weight (kg) 
Louis, 7, 50, 11
Bernard, 5, 80, 13
Vincent, 9, 60, 11
Gwen, 8, 70, 15
"""},
        {"role": "assistant", "content": """Thought: I will use the function CREATE to make this table.
Action: CREATE_START[{"name": ["Louis", "Bernard", "Vincent", "Gwen"], age": [7, 5, 9, 8], "height (cm)": [50, 80, 60, 70], "weight (kg)": [11, 13, 11, 15]}]CREATE_END"""},
        {"role": "user", "content": "Observation: Created table"},
        {"role": "user", "content": ask_prompt + '\n' + "Question: " + question}
    ]
    curTable = pd.DataFrame
    while not finished:
        gens = interface.generate_history(messages=messages, max_tokens=1024)
        output = gens[0]
        print("Output:")
        print(output)
        print("")
        messages.append({"role": "assistant", "content": output})
        curAction = output[output.find("Action: ") + len("Action: "):]
        if "FINISH" in curAction:
            finished = True
            expression = curAction[curAction.find("FINISH_START[") + len("FINISH_START["):curAction.find("]FINISH_END")]
            try:
                curResponse["received"] = expression
            except:
                print("#################################################")
                print("Finish Error")
                print("Question " + str(counter))
                print(output)
                curResponse["received"] = -987654321
        else:
            evaled = None
            for function in libraryFunctions.keys():
                if function in curAction:
                    exec(libraryFunctions[function][1] + '\n' + "evaled = " + function + '(curTable, """' + curAction + '""")')
                    print(curTable)
                    break
            evaled = str(evaled)
            messages.append({"role": "user", "content": "Observation: " + evaled})
            print("Observation: " + evaled)
    for message in messages:
        print(message["role"])
        print(message["content"])
        print("")


[55]


Going through math questions:   0%|          | 0/1 [00:00<?, ?it/s]

Output:
Thought: To determine the shortest penguin, I need to analyze the height column of the table. I will first create the initial table with the provided data and then add the new penguin, James. After that, I will filter the table to find the penguin with the minimum height.

Action: CREATE_START[{"name": ["Louis", "Bernard", "Vincent", "Gwen"], "age": [7, 5, 9, 8], "height (cm)": [50, 80, 60, 70], "weight (kg)": [11, 13, 11, 15]}]CREATE_END

<class 'pandas.core.frame.DataFrame'>
Observation: Created table
Output:
Thought: Now that the initial table is created, I will add the new penguin, James, to the table. After that, I will find the shortest penguin by filtering the height column.

Action: CREATE_START[{"name": ["Louis", "Bernard", "Vincent", "Gwen", "James"], "age": [7, 5, 9, 8, 12], "height (cm)": [50, 80, 60, 70, 90], "weight (kg)": [11, 13, 11, 15, 12]}]CREATE_END

<class 'pandas.core.frame.DataFrame'>
Observation: Created table
Output:
Thought: The table now includes the 

TypeError: argument of type 'pandas._libs.properties.AxisProperty' is not iterable